In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.linear_model import Ridge
from sklearn.utils import check_random_state
from scipy.linalg import qr
from sklearn.metrics.pairwise import rbf_kernel  
import time
import pandas as pd

# For reproducibility. I.e. when I run multiple times to get same regression result
np.random.seed(42)
def generate_sine_data(n_samples=5000, noise_std=0.1, gap_type='large'):
    """
    Generates noisy sine data with specified gap.
    """
    X = np.linspace(-2 * np.pi, 2 * np.pi, n_samples)
    y = np.sin(X) + np.random.normal(0, noise_std, size=X.shape)

    if gap_type == 'large':
        # Large gap in the center
        gap_mask = (X < -np.pi) | (X > np.pi)
    elif gap_type == 'small':
        # Small gap in the center
        gap_mask = (X < -0.5 * np.pi) | (X > 0.5 * np.pi)
    else:
        raise ValueError("gap_type must be 'large' or 'small'")

    X_train = X[gap_mask]
    y_train = y[gap_mask]

    X_test = X
    y_test = np.sin(X_test)

    return X_train.reshape(-1, 1), y_train, X_test.reshape(-1, 1), y_test

print("Generating datasets...")
X_train1, y_train1, X_test1, y_test1 = generate_sine_data(gap_type='large')
X_train2, y_train2, X_test2, y_test2 = generate_sine_data(gap_type='small')
print("Datasets generated.\n")

def plot_datasets():
    plt.figure(figsize=(14, 6))
    # Dataset 1: Large Gap
    plt.subplot(1, 2, 1)
    plt.scatter(X_train1, y_train1, color='blue', alpha=0.3, s=10, label='Training Data')
    plt.plot(X_test1, np.sin(X_test1), color='green', linewidth=2, label='True Function')
    plt.title('Dataset 1: Large Gap', fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('y', fontsize=12)
    plt.legend()
    # Dataset 2: Small Gap
    plt.subplot(1, 2, 2)
    plt.scatter(X_train2, y_train2, color='red', alpha=0.3, s=10, label='Training Data')
    plt.plot(X_test2, np.sin(X_test2), color='green', linewidth=2, label='True Function')
    plt.title('Dataset 2: Small Gap', fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('y', fontsize=12)
    plt.legend()

    plt.tight_layout()
    plt.savefig('datasets.png', dpi=300)
    plt.close()
    print("Plot 'datasets.png' saved.\n")

plot_datasets()
# Method 1: Exact GP and KRR
def method1_gp_krr(X_train, y_train, X_test, y_test, noise=0.1, kernel_length=1.0, alpha=0.01):
    times = {}

    # Gaussian Process Regression
    kernel = RBF(length_scale=kernel_length) + WhiteKernel(noise_level=noise**2)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0, normalize_y=True)

    start_time = time.time()
    gp.fit(X_train, y_train)
    times['GP_fit'] = time.time() - start_time

    start_time = time.time()
    gp_mean, gp_std = gp.predict(X_test, return_std=True)
    times['GP_predict'] = time.time() - start_time

    # Kernel Ridge Regression
    krr = KernelRidge(kernel='rbf', gamma=1.0 / (2 * kernel_length**2), alpha=alpha)

    start_time = time.time()
    krr.fit(X_train, y_train)
    times['KRR_fit'] = time.time() - start_time

    start_time = time.time()
    krr_pred = krr.predict(X_test)
    times['KRR_predict'] = time.time() - start_time

    return gp_mean, gp_std, krr_pred, times

print("Performing Method 1: Exact GP and KRR...")
gp_mean1, gp_std1, krr_pred1, times1 = method1_gp_krr(X_train1, y_train1, X_test1, y_test1)
gp_mean2, gp_std2, krr_pred2, times2 = method1_gp_krr(X_train2, y_train2, X_test2, y_test2)
print("Method 1 completed.\n")

def plot_method1():
    plt.figure(figsize=(14, 12))

    # Dataset 1
    plt.subplot(2, 1, 1)
    plt.scatter(X_train1, y_train1, color='blue', alpha=0.3, s=10, label='Training Data')
    plt.plot(X_test1, np.sin(X_test1), color='green', linewidth=2, label='True Function')
    plt.plot(X_test1, gp_mean1, color='red', linewidth=1.5, label='GP Mean')
    plt.fill_between(X_test1.ravel(),
                     gp_mean1 - 2 * gp_std1,
                     gp_mean1 + 2 * gp_std1,
                     color='red', alpha=0.2, label='GP 95% CI')
    plt.plot(X_test1, krr_pred1, color='purple', linewidth=1.5, label='KRR Prediction')
    plt.title('Method 1: Exact GP and KRR on Dataset 1 (Large Gap)', fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('y', fontsize=12)
    plt.legend()

    # Dataset 2
    plt.subplot(2, 1, 2)
    plt.scatter(X_train2, y_train2, color='blue', alpha=0.3, s=10, label='Training Data')
    plt.plot(X_test2, np.sin(X_test2), color='green', linewidth=2, label='True Function')
    plt.plot(X_test2, gp_mean2, color='red', linewidth=1.5, label='GP Mean')
    plt.fill_between(X_test2.ravel(),
                     gp_mean2 - 2 * gp_std2,
                     gp_mean2 + 2 * gp_std2,
                     color='red', alpha=0.2, label='GP 95% CI')
    plt.plot(X_test2, krr_pred2, color='purple', linewidth=1.5, label='KRR Prediction')
    plt.title('Method 1: Exact GP and KRR on Dataset 2 (Small Gap)', fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('y', fontsize=12)
    plt.legend()

    plt.tight_layout()
    plt.savefig('method1_gp_krr.png', dpi=300)
    plt.close()
    print("Plot 'method1_gp_krr.png' saved.\n")

plot_method1()

# Method 2: Random Fourier Features (RFF)
def generate_rff_features(X, m, sigma=1.0, random_state=None):
    rng = check_random_state(random_state)
    d = X.shape[1]
    omega = rng.normal(loc=0, scale=1.0 / sigma, size=(d, m))
    b = rng.uniform(0, 2 * np.pi, size=m)
    Z_cos = np.sqrt(2.0 / m) * np.cos(X @ omega + b)
    Z_sin = np.sqrt(2.0 / m) * np.sin(X @ omega + b)
    Z = np.hstack([Z_cos, Z_sin])
    return Z

def method2_rff(X_train, y_train, X_test, y_test, m=500, sigma=1.0, alpha=1e-3, random_state=42):
    times = {}

    # Generate RFF for training and test data
    start_time = time.time()
    Z_train = generate_rff_features(X_train, m, sigma, random_state)
    Z_test = generate_rff_features(X_test, m, sigma, random_state)
    times['RFF_feature_generation'] = time.time() - start_time

    # Ridge Regression in feature space
    start_time = time.time()
    ridge = Ridge(alpha=alpha, fit_intercept=False)
    ridge.fit(Z_train, y_train)
    times['Ridge_fit'] = time.time() - start_time

    start_time = time.time()
    y_pred = ridge.predict(Z_test)
    times['Ridge_predict'] = time.time() - start_time

    # Approximate Predictive Standard Deviation (simplified)
    y_std = np.sqrt(alpha) * np.ones_like(y_pred)

    return y_pred, y_std, times

print("Performing Method 2: Random Fourier Features (RFF)...")
rff_pred1, rff_std1, times_rff1 = method2_rff(X_train1, y_train1, X_test1, y_test1)
rff_pred2, rff_std2, times_rff2 = method2_rff(X_train2, y_train2, X_test2, y_test2)
print("Method 2 completed.\n")

def plot_method2():
    plt.figure(figsize=(14, 12))

    # Dataset 1
    plt.subplot(2, 1, 1)
    plt.scatter(X_train1, y_train1, color='blue', alpha=0.3, s=10, label='Training Data')
    plt.plot(X_test1, np.sin(X_test1), color='green', linewidth=2, label='True Function')
    plt.plot(X_test1, rff_pred1, color='orange', linewidth=1.5, label='RFF Prediction')
    plt.fill_between(X_test1.ravel(),
                     rff_pred1 - 2 * rff_std1,
                     rff_pred1 + 2 * rff_std1,
                     color='orange', alpha=0.2, label='RFF Approx. 95% CI')
    plt.title('Method 2: RFF on Dataset 1 (Large Gap)', fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('y', fontsize=12)
    plt.legend()

    # Dataset 2
    plt.subplot(2, 1, 2)
    plt.scatter(X_train2, y_train2, color='blue', alpha=0.3, s=10, label='Training Data')
    plt.plot(X_test2, np.sin(X_test2), color='green', linewidth=2, label='True Function')
    plt.plot(X_test2, rff_pred2, color='orange', linewidth=1.5, label='RFF Prediction')
    plt.fill_between(X_test2.ravel(),
                     rff_pred2 - 2 * rff_std2,
                     rff_pred2 + 2 * rff_std2,
                     color='orange', alpha=0.2, label='RFF Approx. 95% CI')
    plt.title('Method 2: RFF on Dataset 2 (Small Gap)', fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('y', fontsize=12)
    plt.legend()

    plt.tight_layout()
    plt.savefig('method2_rff.png', dpi=300)
    plt.close()
    print("Plot 'method2_rff.png' saved.\n")

plot_method2()

# Method 3: Performer's Random Feature Approach
def generate_orthogonal_rff_features(X, m, sigma=1.0, random_state=None):
    rng = check_random_state(random_state)
    d = X.shape[1]
    if m % 2 != 0:
        m += 1
    omega = rng.normal(loc=0, scale=1.0 / sigma, size=(d, m))
    # Orthogonalize using QR decomposition
    omega, _ = qr(omega, mode='economic')
    b = rng.uniform(0, 2 * np.pi, size=m)
    Z_cos = np.sqrt(2.0 / m) * np.cos(X @ omega + b)
    Z_sin = np.sqrt(2.0 / m) * np.sin(X @ omega + b)
    Z = np.hstack([Z_cos, Z_sin])
    return Z

def method3_performer(X_train, y_train, X_test, y_test, m=500, sigma=1.0, alpha=1e-3, random_state=42):
    times = {}

    # Generate Orthogonal RFF for training and test data
    start_time = time.time()
    Z_train = generate_orthogonal_rff_features(X_train, m, sigma, random_state)
    Z_test = generate_orthogonal_rff_features(X_test, m, sigma, random_state)
    times['Performer_feature_generation'] = time.time() - start_time

    # Ridge Regression in feature space
    start_time = time.time()
    ridge = Ridge(alpha=alpha, fit_intercept=False)
    ridge.fit(Z_train, y_train)
    times['Ridge_fit'] = time.time() - start_time

    start_time = time.time()
    y_pred = ridge.predict(Z_test)
    times['Ridge_predict'] = time.time() - start_time

    # Approximate Predictive Standard Deviation (simplified)
    y_std = np.sqrt(alpha) * np.ones_like(y_pred)

    return y_pred, y_std, times

print("Performing Method 3: Performer's Random Feature Approach...")
performer_pred1, performer_std1, times_perf1 = method3_performer(X_train1, y_train1, X_test1, y_test1)
performer_pred2, performer_std2, times_perf2 = method3_performer(X_train2, y_train2, X_test2, y_test2)
print("Method 3 completed.\n")

def plot_method3():
    plt.figure(figsize=(14, 12))

    # Dataset 1
    plt.subplot(2, 1, 1)
    plt.scatter(X_train1, y_train1, color='blue', alpha=0.3, s=10, label='Training Data')
    plt.plot(X_test1, np.sin(X_test1), color='green', linewidth=2, label='True Function')
    plt.plot(X_test1, performer_pred1, color='magenta', linewidth=1.5, label='Performer Prediction')
    plt.fill_between(X_test1.ravel(),
                     performer_pred1 - 2 * performer_std1,
                     performer_pred1 + 2 * performer_std1,
                     color='magenta', alpha=0.2, label='Performer Approx. 95% CI')
    plt.title('Method 3: Performer\'s Orthogonal RFF on Dataset 1 (Large Gap)', fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('y', fontsize=12)
    plt.legend()

    # Dataset 2
    plt.subplot(2, 1, 2)
    plt.scatter(X_train2, y_train2, color='blue', alpha=0.3, s=10, label='Training Data')
    plt.plot(X_test2, np.sin(X_test2), color='green', linewidth=2, label='True Function')
    plt.plot(X_test2, performer_pred2, color='magenta', linewidth=1.5, label='Performer Prediction')
    plt.fill_between(X_test2.ravel(),
                     performer_pred2 - 2 * performer_std2,
                     performer_pred2 + 2 * performer_std2,
                     color='magenta', alpha=0.2, label='Performer Approx. 95% CI')
    plt.title('Method 3: Performer\'s Orthogonal RFF on Dataset 2 (Small Gap)', fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('y', fontsize=12)
    plt.legend()

    plt.tight_layout()
    plt.savefig('method3_performer.png', dpi=300)
    plt.close()
    print("Plot 'method3_performer.png' saved.\n")

plot_method3()

# ----------------------------
# Compute Condition Numbers
# ----------------------------

def compute_condition_numbers(
    X_train1, X_train2,
    y_train1, y_train2,
    noise=0.1, kernel_length=1.0,
    alpha_krr=0.01,
    alpha_rff=1e-3,
    m=500,
    sigma=1.0,
    random_state=42
):
    """
    Compute the condition numbers for:
    - Exact GP: condition number of (K + noise^2 I)
    - Kernel Ridge: condition number of (K + alpha_krr I)
    - RFF: condition number of (Z^T Z + alpha_rff I)
    - Performer: condition number of (Z^T Z + alpha_rff I)
    for both datasets.
    """
    cond_numbers = {
        'Exact GP': [],
        'Kernel Ridge': [],
        'RFF': [],
        'Performer': []
    }

    # --- For dataset 1
    n1 = len(X_train1)
    K_gp_1 = rbf_kernel(X_train1, X_train1, gamma=1.0/(2*kernel_length**2))
    K_gp_1 += (noise**2) * np.eye(n1)
    cond_gp_1 = np.linalg.cond(K_gp_1)

    K_krr_1 = rbf_kernel(X_train1, X_train1, gamma=1.0/(2*kernel_length**2))
    K_krr_1 += alpha_krr * np.eye(n1)
    cond_krr_1 = np.linalg.cond(K_krr_1)

    Z_rff_1 = generate_rff_features(X_train1, m, sigma, random_state)
    A_rff_1 = Z_rff_1.T @ Z_rff_1 + alpha_rff * np.eye(Z_rff_1.shape[1])
    cond_rff_1 = np.linalg.cond(A_rff_1)

    Z_perf_1 = generate_orthogonal_rff_features(X_train1, m, sigma, random_state)
    A_perf_1 = Z_perf_1.T @ Z_perf_1 + alpha_rff * np.eye(Z_perf_1.shape[1])
    cond_perf_1 = np.linalg.cond(A_perf_1)

    cond_numbers['Exact GP'].append(cond_gp_1)
    cond_numbers['Kernel Ridge'].append(cond_krr_1)
    cond_numbers['RFF'].append(cond_rff_1)
    cond_numbers['Performer'].append(cond_perf_1)

    # --- For dataset 2
    n2 = len(X_train2)
    K_gp_2 = rbf_kernel(X_train2, X_train2, gamma=1.0/(2*kernel_length**2))
    K_gp_2 += (noise**2) * np.eye(n2)
    cond_gp_2 = np.linalg.cond(K_gp_2)

    K_krr_2 = rbf_kernel(X_train2, X_train2, gamma=1.0/(2*kernel_length**2))
    K_krr_2 += alpha_krr * np.eye(n2)
    cond_krr_2 = np.linalg.cond(K_krr_2)

    Z_rff_2 = generate_rff_features(X_train2, m, sigma, random_state)
    A_rff_2 = Z_rff_2.T @ Z_rff_2 + alpha_rff * np.eye(Z_rff_2.shape[1])
    cond_rff_2 = np.linalg.cond(A_rff_2)

    Z_perf_2 = generate_orthogonal_rff_features(X_train2, m, sigma, random_state)
    A_perf_2 = Z_perf_2.T @ Z_perf_2 + alpha_rff * np.eye(Z_perf_2.shape[1])
    cond_perf_2 = np.linalg.cond(A_perf_2)

    cond_numbers['Exact GP'].append(cond_gp_2)
    cond_numbers['Kernel Ridge'].append(cond_krr_2)
    cond_numbers['RFF'].append(cond_rff_2)
    cond_numbers['Performer'].append(cond_perf_2)

    return cond_numbers

print("Computing condition numbers for all methods...")
cond_nums = compute_condition_numbers(
    X_train1, X_train2,
    y_train1, y_train2,
    noise=0.1,
    kernel_length=1.0,
    alpha_krr=0.01,
    alpha_rff=1e-3,
    m=500,
    sigma=1.0,
    random_state=42
)

df_cond = pd.DataFrame(cond_nums, index=["Dataset 1 (Large Gap)", "Dataset 2 (Small Gap)"])
print("Condition Numbers:")
print(df_cond, "\n")
def summarize_methods():
    from sklearn.metrics import mean_squared_error

    methods = ['Exact GP', 'Kernel Ridge', 'RFF', 'Performer']
    datasets = ['Dataset 1 (Large Gap)', 'Dataset 2 (Small Gap)']
    mse = {
        'Exact GP': [mean_squared_error(y_test1, gp_mean1),
                     mean_squared_error(y_test2, gp_mean2)],
        'Kernel Ridge': [mean_squared_error(y_test1, krr_pred1),
                         mean_squared_error(y_test2, krr_pred2)],
        'RFF': [mean_squared_error(y_test1, rff_pred1),
                mean_squared_error(y_test2, rff_pred2)],
        'Performer': [mean_squared_error(y_test1, performer_pred1),
                      mean_squared_error(y_test2, performer_pred2)]
    }

    cpu_times = {
        'Exact GP Fit (s)': [times1['GP_fit'], times2['GP_fit']],
        'Exact GP Predict (s)': [times1['GP_predict'], times2['GP_predict']],
        'KRR Fit (s)': [times1['KRR_fit'], times2['KRR_fit']],
        'KRR Predict (s)': [times1['KRR_predict'], times2['KRR_predict']],
        'RFF Feature Gen (s)': [times_rff1['RFF_feature_generation'], times_rff2['RFF_feature_generation']],
        'RFF Ridge Fit (s)': [times_rff1['Ridge_fit'], times_rff2['Ridge_fit']],
        'RFF Predict (s)': [times_rff1['Ridge_predict'], times_rff2['Ridge_predict']],
        'Performer Feature Gen (s)': [times_perf1['Performer_feature_generation'], times_perf2['Performer_feature_generation']],
        'Performer Ridge Fit (s)': [times_perf1['Ridge_fit'], times_perf2['Ridge_fit']],
        'Performer Predict (s)': [times_perf1['Ridge_predict'], times_perf2['Ridge_predict']],
    }

    df_mse = pd.DataFrame(mse, index=datasets).round(6)
    df_cpu = pd.DataFrame(cpu_times, index=datasets).round(4)
    df_combined = pd.concat([df_mse, df_cpu], axis=1)

    df_cond_renamed = df_cond.rename(columns={
         'Exact GP': 'Exact GP Cond',
         'Kernel Ridge': 'Kernel Ridge Cond',
         'RFF': 'RFF Cond',
         'Performer': 'Performer Cond'
    })
    df_final = pd.concat([df_combined, df_cond_renamed], axis=1)
    df_final.to_csv('comparison_table.csv')

    print("Comparison of Methods (Mean Squared Error, CPU Times, and Condition Numbers):") # For COmmand line debugging
    print(df_final)
    print("\nSummary table saved as 'comparison_table.csv'.\n")

summarize_methods()

print("All plots, tables, and condition number calculations have been sucesfully generated and saved!!")


Generating datasets...
Datasets generated.

Plot 'datasets.png' saved.

Performing Method 1: Exact GP and KRR...
Method 1 completed.

Plot 'method1_gp_krr.png' saved.

Performing Method 2: Random Fourier Features (RFF)...
Method 2 completed.

Plot 'method2_rff.png' saved.

Performing Method 3: Performer's Random Feature Approach...
Method 3 completed.

Plot 'method3_performer.png' saved.

Computing condition numbers for all methods...
Condition Numbers:
                           Exact GP  Kernel Ridge           RFF     Performer
Dataset 1 (Large Gap)  76567.464789  76567.464789  1.604614e+06  2.500501e+06
Dataset 2 (Small Gap)  86482.001239  86482.001239  1.834207e+06  3.751251e+06 

Comparison of Methods (Mean Squared Error, CPU Times, and Condition Numbers):
                       Exact GP  Kernel Ridge       RFF  Performer  \
Dataset 1 (Large Gap)  0.040202      0.066064  0.015719   0.000019   
Dataset 2 (Small Gap)  0.000032      0.000202  0.013026   0.000000   

                 